In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")
network_name = "zutphen_tki_netwerk"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_selectie_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [ ]:
# Create networkanalysis
network = RibasimLumpingNetwork(
    name=network_name, 
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data()
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:

network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151' # duikers voor wijk Zuidwijken
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
    ], 
    node_ids_to_include=[
        # 1455, # extra punt rondom verdeelpunt De Berkel
    ],
    node_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

TODO: Find and create ribasim_edges_gdf between basins

In [ ]:
conn = (network.split_nodes[['mesh1d_nNodes','geometry', 'split_type']]
        .rename(columns={"geometry":"geom_split_node"}))
# check if split_node is used (split_type)
conn = conn[conn['split_type']!='no_split']

In [ ]:
# merge splitnodes add connected edges
conn_ds = conn.merge(
    network.edges_gdf[['basin', 'start_node_no', 'end_node_no','mesh1d_nEdges']],
    left_on='mesh1d_nNodes', 
    right_on='start_node_no'
)
conn_us = conn.merge(
    network.edges_gdf[['basin', 'start_node_no','end_node_no','mesh1d_nEdges']],
    left_on='mesh1d_nNodes', 
    right_on='end_node_no'
)

In [ ]:
conn

In [ ]:
# merge splitnodes with basin DOWNSTREAM
conn_ds = conn_ds.merge(
    network.basins_gdf[['basin', 'geometry']], 
    left_on='basin', 
    right_on='basin'
).rename(columns={"geometry":"geom_basin"})
conn_ds['side'] = 'downstream'

In [ ]:
# merge splitnodes with basin UPSTREAM
conn_us = conn_us.merge(
    network.basins_gdf[['basin', 'geometry']], 
    left_on='basin', 
    right_on='basin'
).rename(columns={"geometry": "geom_basin"})
conn_us['side'] = 'upstream'

In [ ]:
# COMBINE UPSTREAM AND DOWNSTREAM
conn = pd.concat([conn_ds, conn_us])
conn = conn.drop(columns=['geom_split_node','geom_basin'])

In [ ]:
# merge basin connections with nodes
conn_us = conn_us.merge(
    network.nodes_gdf[['mesh1d_nNodes', 'geometry']], 
    left_on='start_node_no', 
    right_on='mesh1d_nNodes', 
    suffixes=('', '_x')
).rename(columns={"geometry":"edge_start_node"}).drop('mesh1d_nNodes_x', axis=1)
conn_us["edge_start_node_x"] = conn_us.edge_start_node.apply(lambda p: p.x)
conn_us["edge_start_node_y"] = conn_us.edge_start_node.apply(lambda p: p.y)
conn_ds = conn_ds.merge(
    network.nodes_gdf[['mesh1d_nNodes', 'geometry']], 
    left_on='end_node_no', 
    right_on='mesh1d_nNodes', 
    suffixes=('', '_x')
).rename(columns={"geometry":"edge_end_node"}).drop('mesh1d_nNodes_x', axis=1)
conn_ds["edge_end_node_x"] = conn_ds.edge_end_node.apply(lambda p: p.x)
conn_ds["edge_end_node_y"] = conn_ds.edge_end_node.apply(lambda p: p.y)

In [ ]:
# merge upstream and downstream connections
conn = conn_us.merge(
    conn_ds.drop(columns=['geom_split_node', 'split_type']), 
    left_on='mesh1d_nNodes',
    right_on='mesh1d_nNodes',
    suffixes=('_us', '_ds')
)

In [ ]:
conn

In [ ]:
# add coordinate in middle of two nodes upstream and downstream of splitpoint
conn['extra_point_x'] = (conn.edge_start_node_x + conn.edge_end_node_x)/2
conn['extra_point_y'] = (conn.edge_start_node_y + conn.edge_end_node_y)/2
conn['extra_point'] = gpd.points_from_xy(conn['extra_point_x'], conn['extra_point_y'])

conn['edge_us'] = conn.apply(lambda row: LineString([row['geom_basin_us'], row['extra_point']]), axis=1)
conn['edge_ds'] = conn.apply(lambda row: LineString([row['extra_point'], row['geom_basin_ds']]), axis=1)
conn['geometry'] = conn.apply(lambda row: LineString([row['geom_basin_us'], row['extra_point'], row['geom_basin_ds']]), axis=1)
conn = gpd.GeoDataFrame(conn, geometry='geometry', crs=28992)

In [ ]:
conn_us = gpd.GeoDataFrame(
    data=(conn[['mesh1d_nNodes', 'split_type', 'edge_us', 'basin_us', 'mesh1d_nEdges_us', 'side_us']]
          .rename(columns={'edge_us': 'geometry', 'basin_us': 'basin', 'side_us': 'side', 'mesh1d_nEdges_us': 'mesh1d_nEdges'})), 
    geometry='geometry', 
    crs=28992
)
conn_ds = gpd.GeoDataFrame(
    data=(conn[['mesh1d_nNodes', 'split_type', 'edge_ds', 'basin_ds', 'mesh1d_nEdges_ds', 'side_ds']]
          .rename(columns={'edge_ds': 'geometry', 'basin_ds': 'basin', 'side_ds': 'side', 'mesh1d_nEdges_ds': 'mesh1d_nEdges'})), 
    geometry='geometry', 
    crs=28992
)

In [ ]:
conn_us.head()

In [ ]:
conn_ds.head()

In [ ]:
# ribasim_edges = pd.concat([conn_us, conn_ds])

In [ ]:
# network.ribasim_edges_gdf = ribasim_edges

Export everything to geopackage

In [ ]:
# network.export_to_geopackage(output_dir=results_dir)

In [ ]:
# network.edges_gdf.basin.unique()